In [1]:
import pandas as pd

In [2]:
# loading data and examinining what information we have
data = pd.read_csv('dentists_wide.csv', index_col = 0)
print(data.columns.values)

['name' 'wkp_name_1' 'wkp_name_2' 'wkp_name_3' 'wkp_name_4' 'wkp_name_5'
 'wkp_name_6' 'wkp_name_7' 'wkp_name_8' 'wkp_name_9' 'wkp_street_1'
 'wkp_street_2' 'wkp_street_3' 'wkp_street_4' 'wkp_street_5'
 'wkp_street_6' 'wkp_street_7' 'wkp_street_8' 'wkp_street_9' 'wkp_psc_1'
 'wkp_psc_2' 'wkp_psc_3' 'wkp_psc_4' 'wkp_psc_5' 'wkp_psc_6' 'wkp_psc_7'
 'wkp_psc_8' 'wkp_psc_9' 'wkp_town_1' 'wkp_town_2' 'wkp_town_3'
 'wkp_town_4' 'wkp_town_5' 'wkp_town_6' 'wkp_town_7' 'wkp_town_8'
 'wkp_town_9' 'wkp_phone_1' 'wkp_phone_2' 'wkp_phone_3' 'wkp_phone_4'
 'wkp_phone_5' 'wkp_phone_6' 'wkp_phone_7' 'wkp_phone_8' 'wkp_phone_9']


For our primary analysis we need only ID and town

In [3]:
# subseting dataset
columns = data.columns.to_series().str.contains('town')
columns[0] = True

data_sub = data[data.columns[columns]]
print(data_sub.columns.values)

['name' 'wkp_town_1' 'wkp_town_2' 'wkp_town_3' 'wkp_town_4' 'wkp_town_5'
 'wkp_town_6' 'wkp_town_7' 'wkp_town_8' 'wkp_town_9']


In [4]:
data_long = pd.melt(data_sub, id_vars = 'name', 
                    value_vars = data_sub.columns.values[1:], 
                    value_name= 'Town')
print(data_long)

                                 name    variable                   Town
0                     MUDr. Amer Abed  wkp_town_1                  Plzeň
1            DDS. Ahmad Amin Abosaleh  wkp_town_1     Velký Třebešov 118
2            MUDr. Fatemeh Aboutorabi  wkp_town_1      Praha 5 - Smíchov
3              MUDr. Marie Abrahamová  wkp_town_1                  Plzeň
4               MUDr. Ahmad Abu Baker  wkp_town_1                Liberec
5                MUDr. František Adam  wkp_town_1                    NaN
6                      MUDr. Jan Adam  wkp_town_1     Jablonec nad Nisou
7                      MDDr. Jan Adam  wkp_town_1                   Brno
8               MUDr. Miluše Adamcová  wkp_town_1         Mladá Boleslav
9               MUDr. Libuše Adamcová  wkp_town_1                    NaN
10               MUDr. Alena Adamcová  wkp_town_1                 Přerov
11               MUDr. Silva Adamcová  wkp_town_1                    NaN
12             MUDr. Markéta Adamcová  wkp_town_1  

In [5]:
data_long = data_long.dropna()
data_long.loc[:,'Town'] = data_long.loc[:,'Town'].str.replace('\d+', '')
data_long.loc[:,'Town'] = data_long.loc[:,'Town'].str.strip()
data_long.loc[:,'Town'][data_long.loc[:,'Town'].str.contains('Praha')] = 'Praha'
print(data_long)

                                       name    variable                   Town
0                           MUDr. Amer Abed  wkp_town_1                  Plzeň
1                  DDS. Ahmad Amin Abosaleh  wkp_town_1         Velký Třebešov
2                  MUDr. Fatemeh Aboutorabi  wkp_town_1                  Praha
3                    MUDr. Marie Abrahamová  wkp_town_1                  Plzeň
4                     MUDr. Ahmad Abu Baker  wkp_town_1                Liberec
6                            MUDr. Jan Adam  wkp_town_1     Jablonec nad Nisou
7                            MDDr. Jan Adam  wkp_town_1                   Brno
8                     MUDr. Miluše Adamcová  wkp_town_1         Mladá Boleslav
10                     MUDr. Alena Adamcová  wkp_town_1                 Přerov
12                   MUDr. Markéta Adamcová  wkp_town_1                Polička
13                     MUDr. Ivana Adamcová  wkp_town_1  Jablonné v Podještědí
14                  MDDr. Adéla Adamczyková  wkp_tow

In [6]:
bridge = pd.read_csv("TOWN_BRICK_REGION.csv")
print(bridge.head())

      NAZEV             OKRES                  KRAJ
0  Abertamy      Karlovy Vary      Karlovarský kraj
1    Adamov        Kutná Hora      Středočeský kraj
2    Adamov  České Budějovice        Jihočeský kraj
3    Adamov           Blansko     Jihomoravský kraj
4  Adršpach            Náchod  Královéhradecký kraj


In [7]:
data_merged = pd.merge(data_long, bridge, how= 'left', left_on= 'Town', right_on = 'NAZEV')
print(data_merged)

                                   name    variable                   Town  \
0                       MUDr. Amer Abed  wkp_town_1                  Plzeň   
1              DDS. Ahmad Amin Abosaleh  wkp_town_1         Velký Třebešov   
2              MUDr. Fatemeh Aboutorabi  wkp_town_1                  Praha   
3                MUDr. Marie Abrahamová  wkp_town_1                  Plzeň   
4                 MUDr. Ahmad Abu Baker  wkp_town_1                Liberec   
5                        MUDr. Jan Adam  wkp_town_1     Jablonec nad Nisou   
6                        MDDr. Jan Adam  wkp_town_1                   Brno   
7                 MUDr. Miluše Adamcová  wkp_town_1         Mladá Boleslav   
8                  MUDr. Alena Adamcová  wkp_town_1                 Přerov   
9                MUDr. Markéta Adamcová  wkp_town_1                Polička   
10                 MUDr. Ivana Adamcová  wkp_town_1  Jablonné v Podještědí   
11              MDDr. Adéla Adamczyková  wkp_town_1             

In [54]:
# deleting parts of the town
strings_1 = ['Pardubice',
            'Havířov',
            'Brno',
            'Ostrava',
            'Plzeň',
            'Karviná',
            'Litoměřice',
            'Třinec',
            'Klatovy',
            'Olomouc',
            'Hranice',
            'Kutná Hora',
            'Karlovy Vary',
            'Říčany',
            'Zruč',
            'Třemošná',
            'Bohumín',
            'Hradec Králové',
            'Jičín',
            'Domažlice',
            'Mariánské Údolí',
            'Opava',
            'Rájec',
            'Herálec',
            'Poděbrady',
            'Přerov',
            'Benešov',
            'Kolín',
            'Liberec',
            'Rokycany',
            'Chrudim',
            'Vodňany',
            'Orlová',
            'Jaroměř',
            'Strakonice',
            'Neratovice',
            'Hutisko',
            'Zlín',
            'Klatovy',
            'Zlatníky',
            'Straškov',
            'Uherské Hradiště',
            'Trutnov',
            'Jindřichův Hradec',
            'Valašské Meziříčí',
            'Litultovice',
            'Hvozd',
            'Valašská Polanka',
            'Krhanice',
            'Záhřeb',
            'Adršpach',
            'Vysoké Mýto']

for i in strings_1:
    data_long.loc[:,'Town'][data_long.loc[:,'Town'].str.contains(i)] = i


# other corrections
string_2_in =  ['Frýdek - Místek',
                'Frýdek- Místek',
                'Frýdek Místek',
                'Kolán',
                'Havl. Brod',
                'LIberec',
                'Plze',
                'Karlových Var',
                'Uherského Hradiště',
                'ŽDÍREC NAD DOUBRAVOU',
                'LItoměřice',
                'Brandýs nad Labem - Stará Boleslav',
                'Hlinsko v Čechách',
                'Zbýšov u Brna',
                'Mikulov na Moravě',
                'Lom u Mostu',
                'Bystřice nad Olší',
                'Planá u Mariánských Lázní',
                'Petrovice u Ústí nad Labem',
                'Hustopeče u Brna',
                'Chlumčany u Přeštic',
                'Jesenice u Rakovníka',
                'Šlapanice u Brna',
                'Ostrov nad Ohří',
                'Trstěnice u Litomyšle',
                'Roztoky u Prahy',
                'Zruč',
                'Nové Sedlo u Lokte',
                'Hořice v Podkrkonoší',
                'Bečov u Mostu',
                'Horní Těrlicko',
                'Šenov u Ostravy',
                'Meziboří u Litvínova',
                'Brandýs nad Labem',
                'Studenec u Horek',
                'Mikulovice u Znojma',
                'Chrást u Plzně',
                'Petřvald u Karviné',
                'Mariánské Údolí',
                'Bystré u Poličky',
                'Milovice nad Labem',
                'Lužná v Čechách',
                'Kostelec u Jihlavy',
                'Šumvald u Uničova',
                'Krásná Lípa u Rumburka',
                'Věkoše',
                'Studnice u Náchoda',
                'Habartice u Frýdlantu',
                'Jindřichov u Šumperka',
                'Zastávka u Brna',
                'Staré Město pod Sněžníkem',
                'Holice v Čechách',
                'Březová u Sokolova',
                'Bohdalice',
                'Bělá nad Radbůzou',
                'Jesenice u Prahy',
                'Újezd u Valašských Klobouk',
                'Dvůr Králové',
                'Loučka u Valašského Meziříčí',
                'Staré Město č.',
                'Zeleneč v Čechách',
                'Frýdlant v Čechách',
                'Krnov, Pod Cilínem',
                'Studná',
                'Králův Dvůr u Berouna',
                'Hať u Hlučína',
                'Petrovice u Karviné č.',
                'Meziměstí u Broumova',
                'Rudník u Vrchlabí',
                'Hutisko',
                'Štítná nad Vláří',
                'Kobylí na Moravě',
                'Opařany č.p.',
                'Sněžné na Moravě',
                'Klašterec nad Ohří',
                'Mokrá',
                'Zlatníky',
                'Straškov',
                'Teplá u Toužimě',
                'Košťany u Teplic',
                'Jílové u Děčína',
                'Morkovice',
                'Petrovice u Sedlčan',
                'Záhřeb',
                'Zábřeh na Moravě',
                'Rudná u Prahy',
                'Troubky nad Bečvou',
                'Vedryně',
                'Brozany',
                'Bor u Tachova',
                'Rychvald u Karviné',
                'Hošťálková u Vsetína',
                'Černožice nad Labem',
                'Ústí nad Labem-Klíše',
                'Dubí u Teplic',
                'Boršice u Buchlovic',
                'Podolí u Brna',
                'Kunžák',
                'Višňové u Znojma',
                'Harrachov v Krkonoších',
                'Kostelec nad Černými Lesy',
                'Javorník u Jeseníku',
                'Olešnice na Moravě',
                'Černovice u Tábora',
                'Čepí č.p.',
                'Proseč u Skutče',
                'Bernartice u Milevska',
                'Lipová - lázně',
                'Janovice č.',
                'Křenovice u Slavkova',
                'Lučina č.',
                'Oskava č.p.',
                'Rousínov u Vyškova',
                'Dolní Sloupnice',
                'Hovorany č.',
                'Jakubčovice',
                'Kamenice n.Lipou',
                'Jablůnka nad Bečvou',
                'Dolní Bečva č.',
                'Svatobořice Mistřín',
                'Miřetice u Hlinska',
                'Písek u Jablunkova',
                'Morkovice - Slížany',
                'LItvínov',
                'Kojetín I - Město',
                'Sedlec - Prčice',
                'Mýto v Čechách',
                'Březolupy č.p.',
                'Kamenice u Jihlavy',
                'Kralupy nad Vltavou-Lobeček',
                'Dolní Újezd u Lipníka nad Bečvou',
                'Lednice na Moravě',
                'Novosedly na Moravě',
                'Zvole u Zábřeha',
                'Vratislavice nad Nisou',
                'Rosice u Brna',
                'Lípa č.p.',
                'Sloup v Moravském Krasu',
                'Děčín  - Nové Město',
                'Ořechov u Brna',
                'Kokory č.',
                'Blatnice pod svatým Antonínkem',
                'Lomnice u Tišnova',
                'Sušice II',
                'Lužice u Hodonína',
                'Čistá u Rakovníka',
                'Hrádek nad Olší',
                'Nezamyslice u Prostějova',
                'Návsí u Jablunkova',
                'Opatov v Čechách',
                'Třebíč - Zámostí',
                'Stará Boleslav',
                'Hlinsko pod Hostýnem č.',
                'Studénka, Butovice',
                'Hrádek u Rokycan',
                'Radnice u Rokycan',
                'Vimperk II',
                'Moravany u Holic',
                'Beroun - Závodí',
                'Osek u Duchcova',
                'Kamenice - Olešovice',
                'Brandýs nad Labem -Stará Boleslav',
                'Chlumec nad Cidlinou IV',
                'Soběslav I',
                'Albrechtice u Českého Těšína',
                'Brumov - Bylnice',
                'Budišov u Třebíče',
                'Holubice - Kozinec',
                'Hostomice pod Brdy',
                'Stránčice',
                'Vraný u Slaného',
                'Chvalkovice v Čechách',
                'Temelín - elektrárna',
                'Vrbátky č.',
                'Brodce nad Jizerou',
                'Záhoří u Písku',
                'Rohozná u Poličky',
                'Borotín u Tábora',
                'Vsetín - Trávníky',
                'Stonava /',
                'Chvalšiny č.p.',
                'Soběšice u Sušic',
                'Činěves č.',
                'Smirady',
                'Luby u Chebu',
                'Dětmarovice č.',
                'Desná v Jizerských horách',
                'Drahotuše',
                'Stráž pod Rálskem',
                'Libina č.p.',
                'Všeruby u Plzně',
                'Hovězí č.p.',
                'Liběšice u Litoměřic',
                'Markvartice u Děčína',
                'Lukavec u Pacova',
                'Borová u Poličky',
                'Bratříkov',
                'Marianské Lázně',
                'Studnice u Vyškova',
                'Brandýs',
                'Hutisko',
                'Jakubčovice',
                'Vratislavice',
                'Boleslav',
                'Smirady',
                'Drahotuše',
                'Alšovice'
                ]

string_2_out = ['Frýdek-Místek',
                'Frýdek-Místek',
                'Frýdek-Místek',
                'Kolín',
                "Havlíčkův Brod",
                "Liberec",
                "Plzeň",
                'Karlovy Vary',
                'Uherské Hradiště',
                'Ždírec nad Doubravou',
                'Litoměřice',
                'Brandýs nad Labem-Stará Boleslav',
                "Hlinsko",
                "Brno",
                "Mikulov",
                "Most",
                "Bystřice",
                "Mariánské Lázně",
                "Ústí nad Labem",
                "Brno",
                "Přeštice",
                "Rakovník",
                "Brno",
                "Ostrov",
                "Litomyšl",
                "Roztoky",
                "Zruč-Senec",
                "Loket",
                "Hořice",
                "Most",
                "Těrlicko",
                "Ostrava",
                "Litvínov",
                "Brandýs",
                "Horky",
                "Znojmo",
                "Plzeň",
                "Karviná",
                "Litvínov",
                "Polička",
                "Nymburk",
                "Rakovník",
                "Jihlava",
                "Uničov",
                "Rumburk",
                "Hradec Králové",
                "Náchod",
                "Frýdlant",
                "Šumperk",
                "Brno",
                "Jindřichov",
                "Pardubice",
                "Sokolov",
                "Pavlovice",
                "Rozvadov",
                "Praha",
                "Valašské Klobouky",
                "Trutnov",
                "Valašské Meziříčí",
                "Staré Město",
                "Zeleneč",
                "Frýdlant",
                "Krnov",
                "Studná",
                "Beroun",
                "Hlučín",
                "Karviná",
                "Broumov",
                "Vrchlabí",
                "Hutisko",
                "Zlín",
                "Terezín",
                "Opařany",
                "Sněžné",
                "Klášterec nad Ohří",
                "Brno",
                "Zlatníky-Hodkovice",
                "Straškov-Vodochody",
                "Toužim",
                "Teplice",
                "Děčín",
                "Morkovice-Slížany",
                "Sedlčany",
                "Šumperk",
                "Šumperk",
                "Praha",
                "Troubky",
                "Vendryně",
                "Pardubice",
                "Tachov",
                "Karviná",
                "Vsetín",
                "Hradec Králové",
                "Ústí nad Labem",
                "Teplice",
                "Buchlovice",
                "Brno",
                "Jindřichův Hradec",
                "Znojmo",
                "Harrachov",
                "Jevany",
                "Jeseník",
                "Olešnice",
                "Tábor",
                "Čepí",
                "Proseč",
                "Bernartice",
                "Lipová",
                "Janovice",
                "Křenovice",
                "Lučina",
                "Oskava",
                "Rousínov",
                "Sloupnice",
                "Hovorany",
                "Jakubčovice",
                "Kamenice",
                "Jablůnka",
                "Dolní Bečva",
                "Svatobořice-Mistřín",
                "Miřetice",
                "Písek",
                "Morkovice-Slížany",
                "Litvínov",
                "Kojetín",
                "Sedlec-Prčice",
                "Mýto",
                "Březolupy",
                "Kamenice",
                "Kralupy nad Vltavou",
                "Újezd",
                "Lednice",
                "Novosedly",
                "Zvole",
                "Vratislavice",
                "Rosice",
                "Lípa",
                "Sloup",
                "Děčín",
                "Ořechov",
                "Kokory",
                "Blatnice",
                "Lomnice",
                "Sušice",
                "Lužice",
                "Rakovník",
                "Hrádek",
                "Nezamyslice",
                "Návsí",
                "Opatov",
                "Třebíč",
                "Boleslav",
                "Hlinsko",
                "Studénka",
                "Rokycany",
                "Rokycany",
                "Vimperk",
                "Moravany",
                "Beroun",
                "Osek",
                "Kamenice",
                "Brandýs nad Labem",
                "Chlumec nad Cidlinou",
                "Soběslav",
                "Albrechtice",
                "Brumov",
                "Budišov",
                "Holubice",
                "Hostomice",
                "Říčany",
                "Vraný",
                "Chvalkovice",
                "Temelín",
                "Vrbátky",
                "Mladá Boleslav",
                "Záhoří",
                "Rohozná",
                "Borotín",
                "Vsetín",
                "Stonava",
                "Chvalšiny",
                "Soběšice",
                "Činěves",
                "Smirady",
                "Luby",
                "Dětmarovice",
                "Desná",
                "Drahotuše",
                "Stráž",
                "Libina",
                "Všeruby",
                "Hovězí",
                "Liběšice",
                "Markvartice",
                "Lukavec",
                "Borová",
                "Alšovice",
                "Mariánské Lázně",
                "Studnice",
                "Brandýs nad Labem-Stará Boleslav",
                "Rožnov pod Radhoštěm",
                "Jakubčovice nad Odrou",
                "Liberec",
                "Mladá Boleslav",
                "Smidary",
                "Lipník nad Bečvou",
                "Pěnčín"
                ]

for i in range(len(string_2_in)):
    data_long.loc[:,'Town'][data_long.loc[:,'Town'].str.contains(string_2_in[i])] = string_2_out[i]


data_merged = pd.merge(data_long, bridge, how= 'left', left_on= 'Town', right_on = 'NAZEV')

In [55]:
missing = data_merged[data_merged['NAZEV'].isna()]['Town'].unique()
print(len(missing))

1


In [56]:
for i in missing:
    print(i)

Studná


It is not possible to find "Studna" even on google maps, hence this one observation will be dropped

In [ ]:
data_merged = data_merged.dropna()

In [57]:
data_merged['Dentist_count'] = data_merged.groupby('name')['name'].transform('size')
data_merged['Dentist_weight'] = 1/data_merged['Dentist_count']
print(data_merged)

                                   name    variable                   Town  \
0                       MUDr. Amer Abed  wkp_town_1                  Plzeň   
1              DDS. Ahmad Amin Abosaleh  wkp_town_1         Velký Třebešov   
2              MUDr. Fatemeh Aboutorabi  wkp_town_1                  Praha   
3                MUDr. Marie Abrahamová  wkp_town_1                  Plzeň   
4                 MUDr. Ahmad Abu Baker  wkp_town_1                Liberec   
5                        MUDr. Jan Adam  wkp_town_1     Jablonec nad Nisou   
6                        MDDr. Jan Adam  wkp_town_1                   Brno   
7                 MUDr. Miluše Adamcová  wkp_town_1         Mladá Boleslav   
8                  MUDr. Alena Adamcová  wkp_town_1                 Přerov   
9                MUDr. Markéta Adamcová  wkp_town_1                Polička   
10                 MUDr. Ivana Adamcová  wkp_town_1  Jablonné v Podještědí   
11              MDDr. Adéla Adamczyková  wkp_town_1             

In [86]:
dentists_brick = data_merged.groupby('OKRES', as_index = False).agg({"Dentist_weight":"sum"})
dentists_region = data_merged.groupby('KRAJ', as_index = False).agg({"Dentist_weight":"sum"})

dentists_brick = dentists_brick.rename(columns={'Dentist_weight': 'DENTISTS'})
dentists_region = dentists_region.rename(columns={'Dentist_weight': 'DENTISTS'})

dentists_brick = dentists_brick.sort_values("OKRES")
dentists_region = dentists_region.sort_values("KRAJ")

print(dentists_brick.head())
print(dentists_region.head())

         OKRES    DENTISTS
0      Benešov   59.294048
1       Beroun   57.342857
2      Blansko   87.608333
3   Brno-město  613.641667
4  Brno-venkov   81.759524
                 KRAJ     DENTISTS
0  Hlavní město Praha  1825.233333
1   Jihomoravský kraj  1036.176190
2      Jihočeský kraj   428.869048
3    Karlovarský kraj   184.692857
4       Kraj Vysočina   364.566667


In [85]:
population = pd.read_csv('populace_okresy.csv')

population = population.sort_values("OKRES")

print(population.head())

          OKRES  POPULACE
0       Benešov     98708
1        Beroun     93726
55      Blansko    108801
56   Brno-město    380681
57  Brno-venkov    222370


In [91]:
print(set(dentists_brick['OKRES']) - set(population['OKRES']))

{'Hlavní město Praha'}
